<a href="https://colab.research.google.com/github/idutsu/kirikuchikun-ai/blob/main/src/mecab-wiki-a-b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get install -y mecab libmecab-dev mecab-ipadic-utf8
!pip install mecab-python3 pandas
!pip install unidic
!python -m unidic download

In [3]:
import os
import csv
import glob
import html
import json
import pandas as pd
import MeCab
from tqdm import tqdm

In [4]:
input_csv = "/content/drive/MyDrive/kirikuchikun-ai/csv/wikipedia-a-to-b-fixed.csv" #入力ファイル
output_csv = "/content/drive/MyDrive/kirikuchikun-ai/csv/a-b.csv"  # 出力ファイル
#CSVファイルを読み込む
df = pd.read_csv(input_csv)

In [11]:
# MeCabの初期化
tagger = MeCab.Tagger()

# CSVファイルを読み込んで処理
try:
    # 入力CSVを読み込み
    df = pd.read_csv(input_csv)

    # 出力CSVの準備
    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        # ヘッダーを書き込む
        csv_writer.writerow(['noun', 'verb'])

        # tqdmを使って進捗状況を表示
        for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing"):
            phrase = row['Extracted_Phrase']  # Extracted_Phrase列を取得
            noun = ""  # 名詞
            verb = ""  # 動詞

            # MeCabで形態素解析
            tokens = tagger.parse(phrase).splitlines()
            token_list = []
            for token in tokens:
                if '\t' in token:
                    surface, features = token.split('\t')
                    features = features.split(',')
                    pos = features[0]
                    token_list.append((surface, pos))

            # 「助詞：を」を探し、その前後を名詞と動詞として取得
            for i in range(1, len(token_list) - 1):
                if token_list[i][0] == "を" and token_list[i][1] == "助詞":
                    # 「を」の前後が存在するかチェック
                    noun = token_list[i - 1][0] if token_list[i - 1][1] == "名詞" else ""
                    verb = token_list[i + 1][0] if token_list[i + 1][1] == "動詞" else ""
                    break

            # 欠けていても書き込む（空白のまま書き出す）
            csv_writer.writerow([noun, verb])

    print(f"\n処理が完了しました。結果は {output_csv} に保存されています。")

except Exception as e:
    print(f"エラーが発生しました: {e}")



Processing: 100%|██████████| 11518679/11518679 [17:10<00:00, 11175.42it/s]


処理が完了しました。結果は /content/drive/MyDrive/kirikuchikun-ai/csv/a-b.csv に保存されています。
